In [2]:
import pandas as pd
import sqlite3

In [96]:
file_name = 'SYSTEM_DATA_EXPORT_IR PROJECT.xlsx'

data = pd.ExcelFile(file_name)

conn = sqlite3.connect('Supply_chain_db.db')



In [97]:
for sheet_name in data.sheet_names:
    df = pd.read_excel(data, sheet_name=sheet_name)
    # Preprocessing to ensure valid SQL table names and to handle potential issues with data types or missing values
    # Removing spaces and special characters from sheet names for use as table names
    safe_sheet_name = sheet_name.replace(" ", "_").replace("-", "_").replace(".", "_")
    df.to_sql(safe_sheet_name, conn, if_exists='replace', index=False)

# Query to print out all table names in the SQLite database
query = "SELECT name FROM sqlite_master WHERE type='table';"
table_names = pd.read_sql_query(query, conn)
print(table_names)

# Close the connection
conn.close()

1

186

46

26

261

...

13415

470

319

319

                              name
0                         projects
1                              wbs
2                        warehouse
3                        commodity
4                        equipment
5             units_of_measurement
6                            items
7                        items_(2)
8                       inco_terms
9                         supplier
10                   purchaseorder
11          purchaseorder_lineitem
12           shippingcontrolnumber
13  shippingcontrolnumber_packages
14                        expedite
15             warehouse_inventory
16                       fmr_entry
17                     fmr_request
18                _fmr_contractor_


In [99]:
conn = sqlite3.connect('Supply_chain_db.db')


# Query to print out all table names in the SQLite database
query = "SELECT name FROM sqlite_master WHERE type='table';"
table_names_df = pd.read_sql_query(query, conn)

# Print the first 10 rows of each table
for table_name in table_names_df['name']:
    print(f"First 10 rows of table {table_name}:")
    query = f"SELECT * FROM `{table_name}` LIMIT 5;"
    df = pd.read_sql_query(query, conn)
    print(df)
    print("\n")

# Close the connection
conn.close()

First 10 rows of table projects:
   project_id     name                                description code  \
0         156  Redwing  Blurred Reality of Infinite Possibilities   MJ   

                                       Project Title  
0  Redwing :: Blurred Reality of Infinite Possibi...  


First 10 rows of table wbs:
      id  project_id  level code      description      wbs_string  \
0  10146         156      1   MJ          Redwing              MJ   
1  10147         156      2  RAY        Menomonie          MJ-RAY   
2  10148         156      3  RRH       Forest Run      MJ-RAY-RRH   
3  10149         156      4  FAE          Waywood  MJ-RAY-RRH-FAE   
4  10150         156      4  PCA  Clyde Gallagher  MJ-RAY-RRH-PCA   

  parent_string  parent_id               WBS Title wbs.1 wbs.2 wbs.3 wbs.4  \
0          None        NaN           MJ :: Redwing    MJ  None  None  None   
1            MJ    10146.0        RAY :: Menomonie    MJ   RAY  None  None   
2        MJ-RAY    10147.0   

In [100]:
# Connect to the SQLite database
conn = sqlite3.connect('Supply_chain_db.db')

# Fetch warehouse data
#query = "SELECT * FROM warehouse;"



query = "SELECT * FROM warehouse_inventory;"
df = pd.read_sql(query, conn)
display(df.head())
# Close the database connection
conn.close()

# Create a folium map
# Adjust the map center and zoom level as needed based on your data


,id,project_id,warehouse_id,wbs_id,expedite_id,shippingcontrolnumber_id,quantity_available,quantity_inventory,quantity_damaged,quantity_in_transit,grid_location,effective_date,warehouse_transfer_id,warehouse_transfer_package_id,warehouse_transfer_package_item_id,unreceipt_id,item_id_unique,fact_expedite.unit_rate
0,848,156,140,10176,5227.0,11e5d7db-b1b6-4532-8d66-7d83153add84,0.0,0.0,0,1344.0,None,2024-02-10 00:00:00,NaN,NaN,NaN,NaN,292c,393.1128
1,849,156,140,10176,5228.0,11e5d7db-b1b6-4532-8d66-7d83153add84,0.0,0.0,0,1344.0,None,2024-06-16 00:00:00,NaN,NaN,NaN,NaN,292c,393.1128
2,850,156,140,10176,5229.0,11e5d7db-b1b6-4532-8d66-7d83153add84,0.0,0.0,0,1344.0,None,2025-01-19 00:00:00,NaN,NaN,NaN,NaN,292c,393.1128
3,2295,156,166,10176,6673.0,05c33448-e37a-4e97-a49e-5fd1a946e85d,0.0,0.0,0,768.0,None,2024-04-18 00:00:00,NaN,NaN,NaN,NaN,290c,316.1664
4,12144,156,131,10176,8384.0,58b3b3b9-73d9-4b5b-b449-69ed90b43ee8,-8800.0,-8800.0,0,0.0,B-2-8-19,2024-11-18 00:00:00,NaN,NaN,NaN,NaN,282c,178.1051


In [81]:
warehouse_type = set(df['warehouse_type'])
print(warehouse_type)

{'Digital Visionaries', 'Tech Wizards', 'Innovation Pioneers'}


In [26]:
import leafmap.foliumap as leafmap
import folium

In [101]:
m = leafmap.Map(center=[-33.896956, 151.205232], zoom=4)

# Create a folium map
# Adjust the map center and zoom level as needed based on your data
#m = leafmap.Map(center=[40.7128, -74.0060], zoom=10)

# Define colors for different types of warehouses
color_dict = {
    'Digital Visionaries': 'blue',
    'Tech Wizards': 'green',
    'Innovation Pioneers': 'red',
    'default': 'gray',
}



In [39]:
# Add circle points to the map using Folium
for idx, row in df.iterrows():
    # Check if latitude or longitude is NaN and skip if true
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):
        continue

    # Define a HTML popup to display information in a table format
    popup_html = folium.Popup(f"""
    <table style='width:200px'>
        <tr><th colspan='2'>Warehouse Information</th></tr>
        <tr><td><strong>Code:</strong></td><td>{row['warehouse_code']}</td></tr>
        <tr><td><strong>Company:</strong></td><td>{row['warehouse_company']}</td></tr>
        <tr><td><strong>Type:</strong></td><td>{row['warehouse_type']}</td></tr>
    </table>
    """, max_width=250)

    # Get the color based on warehouse type
    color = color_dict.get(row['warehouse_type'], color_dict['default'])

    # Add a circle marker with the defined popup and color
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=8,  # Adjust radius for visibility
        popup=popup_html,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7
    ).add_to(m)
m

In [104]:
# Establish a connection to the SQLite database
conn = sqlite3.connect('Supply_chain_db.db')

# SQL query to join warehouse and warehouse_inventory tables on warehouse_id
query = """
SELECT wi.*, w.warehouse_code, w.warehouse_name, w.latitude, w.longitude
FROM warehouse w
JOIN warehouse_inventory wi ON w.warehouse_id = wi.warehouse_id 
"""
df = pd.read_sql(query, conn)
# Convert effective_date from string to date and format to only keep the date part
df['effective_date'] = pd.to_datetime(df['effective_date']).dt.date

# Close the database connection
conn.close()

In [105]:
display(df.head())

,id,project_id,warehouse_id,wbs_id,expedite_id,shippingcontrolnumber_id,quantity_available,quantity_inventory,quantity_damaged,quantity_in_transit,grid_location,effective_date,warehouse_transfer_id,warehouse_transfer_package_id,warehouse_transfer_package_item_id,unreceipt_id,item_id_unique,fact_expedite.unit_rate,warehouse_code,warehouse_name,latitude,longitude
0,2484,156,154,10192,6862.0,12b5a751-dfa0-4bec-b5d9-0be71f4de8ac,0.0,0.0,0,7.0,None,2024-12-13,NaN,NaN,NaN,NaN,4345e,176028.300,GLITR,Glittering Meadows,-27.468,153.028
1,2486,156,154,10193,6864.0,7c68fcd0-4bdd-41a4-baad-c971ca9bd1f6,0.0,0.0,0,10.0,None,2024-01-19,NaN,NaN,NaN,NaN,4376e,1141893.720,GLITR,Glittering Meadows,-27.468,153.028
2,2495,156,154,10191,6873.0,7e4e9b32-c4b5-4cf6-b7da-2551c0675363,0.0,0.0,0,15.0,None,2025-02-27,NaN,NaN,NaN,NaN,4382e,1370272.464,GLITR,Glittering Meadows,-27.468,153.028
3,2500,156,154,10195,6878.0,337ad396-d58b-46c9-82ac-365f7bc73ffc,0.0,0.0,0,13.0,None,2025-11-29,NaN,NaN,NaN,NaN,4330e,422370.768,GLITR,Glittering Meadows,-27.468,153.028
4,3721,156,154,10192,8099.0,78774857-f913-4a6d-a9cd-8363343fbb9c,0.0,0.0,0,4.0,None,2024-03-20,NaN,NaN,NaN,NaN,299c,159869.600,GLITR,Glittering Meadows,-27.468,153.028


In [108]:
# Process and aggregate inventory data by warehouse, limiting to top 10
def aggregate_inventory_data(group):
    group = group.head(10)  # Limit to top 10 records for simplicity
    html_content = """
    <style>
        table { width: 100%; border-collapse: collapse; }
        th, td { padding: 8px; text-align: left; border-bottom: 1px solid #ddd; }
    </style>
    <table>
        <tr>
            <th>Available</th>
            <th>Inventory</th>
            <th>Damaged</th>
            <th>In Transit</th>
            <th>Effective Date</th>
            <th>Unit Rate</th>
        </tr>
    """
    for _, row in group.iterrows():
        html_content += f"""
        <tr>
            <td>{row['quantity_available']}</td>
            <td>{row['quantity_inventory']}</td>
            <td>{row['quantity_damaged']}</td>
            <td>{row['quantity_in_transit']}</td>
            <td>{row['effective_date']}</td>
            <td>{row['fact_expedite.unit_rate']}</td>
        </tr>
        """
    html_content += "</table>"
    if len(group) >= 10:
        html_content += "<div>Only top 10 items displayed. See more on detailed report.</div>"
    return html_content


inventory_summary = df.groupby(['warehouse_id', 'warehouse_code', 'warehouse_name', 'latitude', 'longitude']).apply(aggregate_inventory_data).reset_index(name='inventory_details')

display(inventory_summary.head(2))

,warehouse_id,warehouse_code,warehouse_name,latitude,longitude,inventory_details
0,129,ENCHT,Enchanted Lagoon,-31.950,115.860,\n <style>\n table { width: 100%; bo...
1,131,SNCOS,Sonic Oasis,-34.928,138.601,\n <style>\n table { width: 100%; bo...


In [109]:

# Initialize a folium map at a central location
map = folium.Map(location=[-25.2744, 133.7751], zoom_start=5)

# Function to create an HTML formatted popup for each warehouse
def create_popup(row):
    return f"""
    <div style='width:360px;'>
        <h4>Warehouse Details</h4>
        <strong>Warehouse Code:</strong> {row['warehouse_code']}<br>
        <strong>Company:</strong> {row['warehouse_name']}<br>
        <h4>Inventory Summary:</h4>
        {row['inventory_details']}
    </div>
    """

# Add markers to the map
for index, row in inventory_summary.iterrows():
    popup = folium.Popup(create_popup(row), max_width=400)
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),  # Placeholder coordinates
        radius=6,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7
    ).add_to(map)

# Save and display the map
map.save('styled_aggregated_warehouse_inventory_map.html')
map

# Using LLM

## Table information

In [2]:
import sqlite3
import pandas as pd

# Function to connect to the SQLite database
def get_db_connection(db_file):
    """Establishes a connection to the specified SQLite database."""
    conn = sqlite3.connect(db_file)
    return conn

# Function to retrieve all tables and their columns from the database
def get_all_table_columns(db_file):
    conn = get_db_connection(db_file)
    cursor = conn.cursor()

    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Dictionary to hold table and columns info
    table_columns = {}

    # Loop through all tables and fetch their respective columns
    for table_name in tables:
        table_name = table_name[0]  # Table name is the first element in the tuple
        cursor.execute(f"PRAGMA table_info(`{table_name}`);")  # Correct quoting for table names
        columns = cursor.fetchall()

        # Extract column names and types
        table_columns[table_name] = [{'name': col[1], 'type': col[2]} for col in columns]

    cursor.close()
    conn.close()
    return table_columns

# Database file path
db_file = 'Supply_chain_db.db'

# Get all table columns
all_table_columns = get_all_table_columns(db_file)

# Print the structure of each table
for table, columns in all_table_columns.items():
    print(f"Table: {table}")
    for column in columns:
        print(f"  Column Name: {column['name']}, Type: {column['type']}")
    print("\n")


Table: projects
  Column Name: project_id, Type: INTEGER
  Column Name: name, Type: TEXT
  Column Name: description, Type: TEXT
  Column Name: code, Type: TEXT
  Column Name: Project Title, Type: TEXT


Table: wbs
  Column Name: id, Type: INTEGER
  Column Name: project_id, Type: INTEGER
  Column Name: level, Type: INTEGER
  Column Name: code, Type: TEXT
  Column Name: description, Type: TEXT
  Column Name: wbs_string, Type: TEXT
  Column Name: parent_string, Type: TEXT
  Column Name: parent_id, Type: REAL
  Column Name: WBS Title, Type: TEXT
  Column Name: wbs.1, Type: TEXT
  Column Name: wbs.2, Type: TEXT
  Column Name: wbs.3, Type: TEXT
  Column Name: wbs.4, Type: TEXT
  Column Name: wbs.5, Type: TEXT
  Column Name: wbs.6, Type: TEXT
  Column Name: wbs.7, Type: TEXT
  Column Name: wbs.8, Type: TEXT


Table: warehouse
  Column Name: warehouse_id, Type: INTEGER
  Column Name: warehouse_name, Type: TEXT
  Column Name: warehouse_address, Type: REAL
  Column Name: warehouse_code, Type: TE

In [7]:
# save the following into a csv file with wbs_string

import pandas as pd

# WBS codes in a single string separated by spaces
wbs_codes_str = """
BR-P4-A1-00-000
BR-P4-A1-10-000
BR-P4-A1-10-010
BR-P4-A1-10-020
BR-P4-A1-10-030
BR-P4-A1-10-040
BR-P4-A1-10-050
BR-P4-A1-10-060
BR-P4-A1-10-070
BR-P4-A1-10-080
BR-P4-A1-10-090
BR-P4-A1-10-100
BR-P4-A1-20-000
BR-P4-A1-20-010
BR-P4-A1-20-020
BR-P4-A1-20-030
BR-P4-A1-20-040
BR-P4-A1-20-050
BR-P4-A1-20-060
BR-P4-A1-20-070
BR-P4-A1-20-080
BR-P4-A1-30-000
BR-P4-A1-30-010
BR-P4-A1-30-020
BR-P4-A1-30-030
BR-P4-A1-30-040
BR-P4-A1-30-050
BR-P4-A1-30-060
BR-P4-A1-30-070
BR-P4-A1-30-080
BR-P4-A1-40-000
BR-P4-A1-40-010
BR-P4-A1-40-020
BR-P4-A1-40-030
BR-P4-A1-40-040
BR-P4-A1-40-050
BR-P4-A1-40-060
BR-P4-A1-40-070
BR-P4-A1-40-080
BR-P4-A1-40-090
BR-P4-A1-40-100
BR-P4-A1-40-110
BR-P4-A1-40-120
BR-P4-A1-40-130
BR-P4-A1-40-140
BR-P4-A1-50-000
BR-P4-A1-50-010
BR-P4-A1-50-020
BR-P4-A1-50-030
BR-P4-A1-50-040
BR-P4-A1-50-050
BR-P4-A1-50-060
BR-P4-A1-50-070
BR-P4-A1-50-080
BR-P4-A1-50-090
BR-P4-A1--
BR-P4-A1-60-000
BR-P4-A1-60-010
BR-P4-A1-60-020
BR-P4-A1-60-030
BR-P4-A1-70-000
BR-P4-A1-70-010
BR-P4-A1-70-020
BR-P4-A1-70-030
BR-P4-A1-70-040
BR-P4-A1-70-050
BR-P4-A1-70-060
BR-P4-A1-70-070
BR-P4-A1-70-080
BR-P4-A1-70-090
BR-P4-A1-70-100
BR-P4-A1-70-110
BR-P4-A1-70-120
BR-P4-A1-70-130
BR-P4-A1-70-140
BR-P4-A1-70-150
BR-P4-A1-70-160
BR-P4-A1-70-170
BR-P4-A1-70-180
BR-P4-A1-80-000
BR-P4-A1-80-010
BR-P4-A1-80-020
BR-P4-A1-80-030
BR-P4-A2-00-000
BR-P4-A2-01-000
BR-P4-A2-01-010
BR-P4-A2-01-020
BR-P4-A2-01-030
BR-P4-A2-01-040
BR-P4-A2-02-000
BR-P4-A2-02-010
BR-P4-A2-02-020
BR-P4-A2-02-030
BR-P4-A2-03-000
BR-P4-A2-03-010
BR-P4-A2-03-020
BR-P4-A2-03-030
BR-P4-A2-03-040
BR-P4-A2-03-050
BR-P4-A2-03-060
BR-P4-A2-03-070
BR-P4-A2-03-080
BR-P4-A2-04-000
BR-P4-A2-04-010
BR-P4-A2-04-020
BR-P4-A2-04-030
BR-P4-A2-04-040
BR-P4-A2-05-000
BR-P4-A2-05-010
BR-P4-A2-05-020
BR-P4-A2-05-030
BR-P4-A2-06-000
BR-P4-A2-06-010
BR-P4-A2-06-020
BR-P4-A2-07-000
BR-P4-A2-07-010
BR-P4-A2-07-020
BR-P4-A2-10-000
BR-P4-A2-10-010
BR-P4-A2-10-020
BR-P4-A2-10-030
BR-P4-A2-10-040
BR-P4-A2-10-050
BR-P4-A2-10-060
BR-P4-A2-10-070
BR-P4-A2-10-080
BR-P4-A2-10-090
BR-P4-A2-10-100
BR-P4-A2-10-110
BR-P4-A2-10-120
BR-P4-A2-12-000
BR-P4-A2-12-010
BR-P4-A2-12-020
BR-P4-A2-13-000
BR-P4-A2-13-010
BR-P4-A2-13-020
BR-P4-A2-14-000
BR-P4-A2-14-010
BR-P4-A2-14-020
BR-P4-A2-15-000
BR-P4-A2-15-010
BR-P4-A2-15-020
BR-P4-A2-16-000
BR-P4-A2-16-010
BR-P4-A2-17-000
BR-P4-A2-17-010
BR-P4-A2-18-000
BR-P4-A2-18-010
BR-P4-A2-19-000
BR-P4-A2-19-010
BR-P4-A2-20-000
BR-P4-A2-20-010
BR-P4-A2-21-000
BR-P4-A2-21-010
BR-P4-A2-22-000
BR-P4-A2-22-010
BR-P4-A2-23-000
BR-P4-A2-23-010
BR-P4-A2-24-000
BR-P4-A2-24-010
BR-P4-A3-00-000
BR-P4-A3-01-000
BR-P4-A3-01-110
BR-P4-A3-02-000
BR-P4-A3-02-210
BR-P4-A3-10-000
BR-P4-A3-10-010
BR-P4-A3-10-020
BR-P4-A3-10-030
BR-P4-A3-10-040
BR-P4-A3-10-050
BR-P4-A3-10-060
BR-P4-A3-10-070
BR-P4-A3-10-080
BR-P4-A3-10-090
BR-P4-A3-10-100
BR-P4-A3-10-110
BR-P4-A3-10-120
BR-P4-A3-10-130
BR-P4-A3-10-140
BR-P4-A3-20-000
BR-P4-A3-20-010
BR-P4-A3-20-020
BR-P4-A3-20-030
BR-P4-A3-20-040
BR-P4-A3-20-050
BR-P4-A3-20-060
BR-P4-A3-20-070
BR-P4-A3-20-080
BR-P4-A3-20-090
BR-P4-A3-20-100
BR-P4-A3-20-110
BR-P4-A3-20-120
BR-P4-A3-20-130
BR-P4-A3-20-140
BR-P4-A3-30-000
BR-P4-A3-30-010
BR-P4-A3-30-020
BR-P4-A3-40-000
BR-P4-A3-40-010
BR-P4-A3-50-000
BR-P4-A3-50-010
BR-P4-A3-60-000
BR-P4-A3-60-010
BR-P4-A3-60-020
BR-P4-A3-70-000
BR-P4-A3-70-010
BR-P4-A4-00-000
BR-P4-A4-01-000
BR-P4-A4-01-010
BR-P4-A4-02-000
BR-P4-A4-02-010
BR-P4-A4-02-020
BR-P4-A4-03-000
BR-P4-A4-03-010
BR-P4-A4-04-000
BR-P4-A4-04-010
BR-P4-A4-04-020
BR-P4-A4-04-030
BR-P4-A4-04-040
BR-P4-A4-04-050
BR-P4-A4-05-000
BR-P4-A4-05-010
BR-P4-A4-05-020
BR-P4-A4-05-030
BR-P4-A4-05-040
BR-P4-A4-06-000
BR-P4-A4-06-010
BR-P4-A4-06-020
BR-P4-A4-07-000
BR-P4-A4-07-010
BR-P4-A4-08-000
BR-P4-A4-08-010
BR-P4-A4-09-000
BR-P4-A4-09-010
BR-P4-A4-10-000
BR-P4-A4-10-010
BR-P4-A4-11-000
BR-P4-A4-11-010
BR-P4-A4-12-000
BR-P4-A4-12-010
BR-P4-A5-00-000
BR-P4-A5-01-000
BR-P4-A5-01-010
BR-P4-A5-02-000
BR-P4-A5-02-010
BR-P4-A5-03-000
BR-P4-A5-03-010
BR-P4-A5-04-000
BR-P4-A5-04-010
BR-P4-A5-05-000
BR-P4-A5-05-010
BR-P4-A5-06-000
BR-P4-A5-06-010
BR-P4-A5-07-000
BR-P4-A5-07-010
BR-P4-A5-08-000
BR-P4-A5-08-010
BR-P4-A5-09-000
BR-P4-A5-09-010
BR-P4-A5-10-000
BR-P4-A5-10-010
BR-P4-A6-00-000
BR-P4-A6-01-000
BR-P4-A6-01-010
BR-P4-A6-02-000
BR-P4-A6-02-010
BR-P4-A6-03-000
BR-P4-A6-03-010
BR-P4-A6-04-000
BR-P4-A6-04-010
BR-P4-A6-05-000
BR-P4-A6-05-010
BR-P4-A6-06-000
BR-P4-A6-06-010
BR-P4-A6-07-000
BR-P4-A6-07-010
BR-P4-A6-08-000
BR-P4-A6-08-010
BR-P4-A6-09-000
BR-P4-A6-09-010
BR-P4-A6-10-000
BR-P4-A6-10-010
BR-P4-A6-11-000
BR-P4-A6-11-010
BR-P4-A6-12-000
BR-P4-A6-12-010
BR-P4-A6-13-000
BR-P4-A6-13-010
BR-P4-A6-14-000
BR-P4-A6-14-010
BR-P4-A7-00-000
BR-P4-A7-01-000
BR-P4-A7-01-010
BR-P4-A7-02-000
BR-P4-A7-02-010
BR-P4-A7-03-000
BR-P4-A7-03-010
BR-P4-A7-04-000
BR-P4-A7-04-010
BR-P4-A7-05-000
BR-P4-A7-05-010
BR-P4-A7-06-000
BR-P4-A7-06-010
BR-P4-A7-07-000
BR-P4-A7-07-010
BR-P4-A7-07-020
BR-P4-A7-07-030
BR-P4-A7-07-040
BR-P4-A7-07-050
BR-P4-A7-07-060
BR-P4-A7-08-000
BR-P4-A7-08-010
BR-P4-A7-09-000
BR-P4-A7-09-010
BR-P4-A7-10-000
BR-P4-A7-10-010
BR-P4-A8-00-000
BR-P4-A8-01-000
BR-P4-A8-01-010
BR-P4-A8-02-000
BR-P4-A8-02-110
BR-P4-A9-00-000
BR-P4-A9-91-000
BR-P4-A9-91-010
BR-P4-A9-91-020
BR-P4-A9-91-030
BR-P4-A9-91-040
BR-P4-A9-91-050
BR-P4-A9-91-060
BR-P4-A9-91-070
BR-P4-A9-91-080
BR-P4-A9-91-090
BR-P4-A9-91-100
BR-P4-A9-91-110
BR-P4-A9-91-120
BR-P4-A9-91-130
BR-P4-A9-91-140
BR-P4-A9-95-000
BR-P4-A9-95-010
BR-P4-A9-95-020
BR-P4-A9-95-030
BR-P4-A9-95-040
BR-P4-A9-95-050
BR-P4-A9-95-060
BR-P4-A9-95-070
"""
descriptions = """
QH Costs
QH Delivery (Staff)
Project (Staff)
Commercial  (Staff)
Engineering Assurance (Staff)
Health & Safety (Staff)
Project Controls (Staff)
Environment (Staff) 
Community and Stakeholder (Staff)
Land (Staff)
Regional Development (Staff)
Quality (Staff)
Project Contractors
Project Support
Commercial Pre-Contracts 
Commercial - Contracts Admin
Project Controls
Legal
Health and Safety Consultants
Engineering Assurance 
Operation and Maintenance Strategy
Engineering 
 Specialist Engineering - External Advisory Services
Specialist Engineering - Owners Engineer
Specialist Engineering - Design for Procurement
Specialist Engineering - FEED
Specialist Engineering - Exploratory Works
Specialist Advisors - Technical Assurance
Connection Application Process/Services
Independent certifier
Environment and Approvals
Technical Advisors - EIS Approvals and Studies
EIS - Engineering Input
EIS - Technical Studies
Exploratory Works Approvals
Main Works - Approvals
Offset Implementation
Sustainability
Office of the Coordinator-General (OCG Costs)
EIS - Delivery
GIS Support
Cultural Heritage - Investigations and Monitoring
Indigenous Land Use Agreement
Compliance Monitoring
Indigenous Partnerships
Community Development and Stakeholder 
Community and Stakeholder Consultants
Community Benefits Program
Sponsorships
Community Engagement Program
Regional Development
Regional Development Accommodation Partners
Regional Development Community Infrastructure
Regional Development - Community Development Initiatives (GRC)
Regional Development - Community Development Initiatives (SRC)
Heath and Safety 
Land Acquisition 
Land Access & Conveyancing
Land Acquisition
Land Access - Specialist Advisors
Indirect 
Project Expenses – General
PPE
Emergency Response Plan
Travel, Accommodation & Disbursements
IT Software, Licenses & Equipment
Training & Development (Delivery)
Mandatory Safety Training 
Site Vehicles and Transport
Project Offices and Facilities 
Portable Long Service Leave Levy
Insurances 
Bid cost reimbursement
Site Equipment & Tooling
Corporate Overheads
Legal Costs
Site Communications
Finance costs (FBT)
Other Staff Costs (memberships, subscriptions)
Ops Readiness & Commissioning 
Op Readiness
Commissioning
Full Load Compliance Testing 
Surface Infrastructure and Enabling 
Contractor Indirect & overheads
Temporary Camps - Indirects and Overheads 
Main Camps - Indirects and Overheads 
Exploratory Works - Contractor Indirect & overheads
Enabling Works  - Contractor Indirect & overheads
Design
Design - Exploratory Works
Design - Enabling Works
Temp Camps Design 
Camps
Temp Camps
Upper Res Camps Hardstand
Lower Res Camps Hardstand
Main Camps Hardstand
Upper Res Camps Install and Operate
Lower Res Camps Install and Operate
Main Camps Install and Operate
Camp Set up 
Geotechnical 
Contractor Indirect & overheads
Geotechnical Studies
Geotechnical Drilling
Geotechnical Service Management 
Temporary Water Infrastructure 
Preliminaries Temporary Water Infrastructure
Temporary Water Infrastructure 
Temporary Water Supply
Communication Infrastructure Upgrade 
Preliminaries Communication Infrastructure Upgrade 
Communication Infrastructure Upgrade 
Power Supply Upgrade
Preliminaries Power Supply Upgrade
Power Supply Upgrade
Yabba Creek Road Option 1
Preliminaries Yabba Ck Rd Option 1
Bridge 1 (on Yabba Ck Rd - Ana Branche No 1)
Bridge 2 (on Yabba Ck Rd - Ana Branche No 2)
Bridge 3  (on Yabba Ck Rd - No 1 Crossing - Neumans Crossing)
Bridge 4 (on Yabba Ck Rd - No 2 Crossing - Elbow Crossing)
Bridge 5 (on Yabba Ck Rd - No 3 Crossing - Andrews Crossing)
Bridge 6 (on Yabba Ck Rd - No ? Crossing - ????)
Bridge 7 (on Yabba Ck Rd - No 4 Crossing - Boardmans Crossing)
Bridge 8 (on Yabba Ck Rd - No 5 Crossing - Friends Crossing)
Bridge 9 (on Yabba Ck Rd - No 6 Crossing - Deep  Crossing)
Bridge 10 (on Yabba Ck Rd - No 7 - Crossing - unknown name)
Upgrade Bella Creek Road intersection
Old Forestry Drive Option 2
Preliminaries Old Forestry Drive Option 2
Old Forestry Drive Option 2 Construction 
Yielo Rd
Preliminaries Upgrade of Yielo Rd
Upgrade of Yielo Rd Construction 
Walkers Top Access Track
Preliminaries Walkers Top Access Track
Walkers Top Access Track Construction
Access Roads - The Borgan
Preliminaries Access Roads - The Borgan
Access Roads - The Borgan
Lower Res Access Road 
Lower Res Access Road 
Lower Res Haul Road - Construct and Maintain
Lower Res Haul Road - Construct and Maintain
Bella Creek Road 
Bella Creek Road 
Yabba Creek Bridge  
Yabba Creek Bridge  
Kingaham Creek Bridge 
Kingaham Creek Bridge 
Road to Lower Inlet / Outlet Structure
Road to Lower Inlet / Outlet Structure
Road to Upper Res/Saddle Dam
Road to Upper Res/Saddle Dam
Ring Road at Upper Res incl. Access to upper intake and Bellmouth 
Ring Road at Upper Res incl. Access to upper intake and Bellmouth 
MAT / ECVT Portal Pad Excavation
MAT / ECVT Portal Pad Excavation
Underground Waterways 
Contractor Indirect & overheads stage 1
Contractor Indirects & overheads
Design 
Design 
North Alignment Waterway
Upper Inlet/Outlet Structure Bellmouth - North
Upper Inlet/Outlet Structure Gate Shaft - North
Lower Inlet/Outlet Structure Bellmouth - North
Lower Inlet/Outlet Structure Gate Shaft Middle to Top - North
Lower Inlet/Outlet Structure Gate Shaft Lower Part/Square Section - North
Low Pressure Tunnel - North
Erection Chamber - Upper - North
Vertical Pressure Shaft - North
Erection Chamber - Lower - North
High Pressure Tunnel - North
Penstock Steel Lined - North
Tailrace Collector Tunnel - North
Tailrace Tunnel - North
Surge Tank - North
South Alignment Waterway
Upper Inlet/Outlet Structure Bellmouth - South
Upper Inlet/Outlet Structure Gate Shaft - South
Lower Inlet/Outlet Structure Bellmouth - South
Lower Inlet/Outlet Structure Gate Shaft Middle to Top - South
Lower Inlet/Outlet Structure Gate Shaft Lower Part/Square Section - South
Low Pressure Tunnel - South
Erection Chamber - Upper - South
Vertical Pressure Shaft - South
Erection Chamber - Lower - South
High Pressure Tunnel - South
Penstock Steel Lined - South
Tailrace Collector Tunnel - South
Tailrace Tunnel - South
Surge Tank - South
Platform and Control Buildings
Upper Inlet/Outlet Platform and Control Building
Lower Inlet/Outlet Platform and Control Building
Lower Inlet Structure Approach 
Lower Inlet/Outlet Structure - Approach
Surge Tanks Permanent access tunnel
Surge Tanks Permanent access tunnel
Assembly Chambers
Upstream Assembly Chambers, Connection Tunnels and Adit
Downstream Assembly Chambers, Connection Tunnels and Adit
Site Rehabilitation 
Site Rehabilitation 
Underground Access & Caverns 
Contractor Indirect & overheads
Contractor Indirects & overheads
Design 
Detail Design - Main Facility
Detail Design - OEM 
Site Clearing 
Site Clearing 
Construction Surface Facilities
Construction Site Facilities -  Platform Areas
Grout plant - mobilise and demobilise
Construction Site Facilities -  Platform 2 - Concrete Plant
Concrete batching plant - mobilise and demobilise
Construction Site Facilities -  Upper inlet / outlet - Concrete Plant
ECTV Exploratory Drilling
Exploratory Tunnel - Surface Works
Exploratory Tunnel - Underground Works
Exploratory Tunnel - Caverns Investigation
Exploratory Tunnel - Management Services
MAT and ECVT
Main Access Tunnel (MAT)
Emergency, Cable and Ventilation Tunnel (ECVT) Completion 
Power Cavern
Power Cavern
Transformer Cavern
Transformer Cavern
IPB Galleries (x 6)
IPB Galleries (x 6)
MAT Tunnels between Power Cavern and Transformer Cavern
MAT Tunnels between Power Cavern and Transformer Cavern
Traffic Control
Traffic Control
Site Rehabilitation 
Site Rehabilitation 
Powerhouse Mech & Elec
Contractor Indirect & overheads
Contractor Indirects & overheads
Design 
Design 
Mech & Elec
Mech & Elec
Power Turbines
Power Turbines
Transformers
Transformers
Hydro-Mechanical Works
Hydro-Mechanical Works
Electrical Install
Electrical Install
Power Evacuation and Grid Connection
Power Evacuation and Grid Connection
Commissioning OEM
Commissioning OEM
Site Rehabilitation 
Site Rehabilitation 
Lower Reservoir
Contractor Indirect & overheads
Contractor Indirect & overheads
Design
Design
Site Clearing 
Site Clearing 
Quarry Development  and Material Operations
Quarry Development  and Material Operations
Diversion and Watering
Diversion and Watering
Foundations
Foundations
Dam Construction
Dam Construction
Spillway
Spillway
Intake Tower
Intake Tower
Outlet Works
Outlet Works
Fishway
Fishway
Over topping Event of the Lower Dam
Over topping Event of the Lower Dam
Traffic Control
Traffic Control
Site Rehabilitation
Site Rehabilitation
Upper Reservoir
Contractor Indirect & overheads
Contractor Indirect & overheads
Design
Design
Site Clearing 
Site Clearing 
Quarry Development  and Material Operations
Quarry Development  and Material Operations
Diversion and Watering
Diversion and Watering
Foundations
Foundations
Dam Construction
Main CFRD Dam
Main Saddle (CFRD) Dam
Secondary Saddle (CFRD) Dam A
Secondary Saddle (CFRD) Dam B1
Secondary Saddle (CFRD) Dam B2
Secondary Saddle (CFRD) Dam No.B3
Emergency Spillway
Emergency Spillway
Traffic Control
Traffic Control
Site Rehabilitation
Site Rehabilitation
Transmissions
Borumba -  Tarong 275kV
Borumba -  Tarong 275kV
Borumba -  Woolooga 275kV
Borumba -  Woolooga 275kV
Contingency & Escalation
Contingency 
P50 Contingency QH Costs
P90 Contingency QH Costs
P50 Contingency Surface Infrastructure and Enabling 
P90 Contingency  Surface Infrastructure and Enabling 
P50 Contingency Underground Waterways 
P90 Contingency Underground Waterways 
P50 Contingency Underground Access & Caverns 
P90 Contingency Underground Access & Caverns 
P50 Contingency Powerhouse Mech & Elec
P90 Contingency Powerhouse Mech & Elec
P50 Contingency Lower Reservoir
P90 Contingency Lower Reservoir
P50 Contingency Upper Reservoir
P90 Contingency Upper Reservoir
Escalation 
Escalation QH Costs
Escalation  Surface Infrastructure and Enabling 
Escalation  Underground Waterways 
Escalation  Underground Access & Caverns 
Escalation  Powerhouse Mech & Elec
Escalation  Lower Reservoir
Escalation  Upper Reservoir
"""



In [10]:
!pwd

/Users/bluekidds/Documents/projects/supply_chain/notebooks


In [12]:
import pandas as pd

def read_and_encode_file(file_path):
    try:
        # Try reading with UTF-8 encoding first
        data = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try with Windows-1252
            data = pd.read_csv(file_path, encoding='windows-1252')
        except UnicodeDecodeError:
            # If Windows-1252 also fails, you can add more encodings to try
            # For example, try with ISO-8859-1
            data = pd.read_csv(file_path, encoding='ISO-8859-1')
    
    return data

# Specify your CSV file path
file_path = '../data/wbs_source.csv'

# Read and encode the file
data_frame = read_and_encode_file(file_path)
print(data_frame.head())


   L3   L3 Desc    L4              L4 Desc    L5     Item  \
0  A1  QH Costs   0.0             QH Costs   0.0   100000   
1  A1  QH Costs  10.0  QH Delivery (Staff)   0.0  A110000   
2  A1  QH Costs  10.0  QH Delivery (Staff)  10.0  A110010   
3  A1  QH Costs  10.0  QH Delivery (Staff)  20.0  A110020   
4  A1  QH Costs  10.0  QH Delivery (Staff)  30.0  A110030   

                     Description  Level         Responsible (From OBS)  \
0                       QH Costs      3                            NaN   
1            QH Delivery (Staff)      4                            NaN   
2                Project (Staff)      5                  Leah Mckenzie   
3            Commercial  (Staff)      5                     Jason Long   
4  Engineering Assurance (Staff)      5  Head of Assets & Engineering    

                  Commentary  Unnamed: 10         wbs-code   Unnamed: 12  \
0       Double up with Staff          NaN  BR-P4-A1-00-000   BRIP4100000   
1  Renumbered from zero to 1        

In [13]:
data_frame.to_csv('wbs_source_formatted.csv', index=False, encoding='utf-8')